# Introduction to Julia

Julia is a recent programming language that aims at combining high performance with ease of use. It is increasingly popular in the optimization community.

## 0. Preliminaries

In this Jupyter notebook, you have access to a structured equivalent of Julia's REPL, i.e. the interactive console. Here, you can divide your code in cells to modify and run each one separately (use `Shift`+`Enter` to run a cell and move to the next one). By default, the output of a cell is the value of its last expression, you can hide it by ending the cell with `;`.

In [ ]:
1 + 2

**This tutorial is quite long, especially the first part. If you want, you can skip directly to part 2 or even 3, and come back to the beginning when you don't know how to do something.**

Although we will introduce some useful commands, we cannot describe all of them. A very good summary can be found at https://juliadocs.github.io/Julia-Cheat-Sheet/, and we will often refer the reader to it.

### Differences with Python

For those who already master Python, here are the key novelties of Julia:
- Even though they are not necessary, types are heavily used to make the code both clearer and faster.
- There are no classes, only "structures" which contain data but no methods.
- Blocks are not based on indentation but delimited by the `end` keyword.
- Array indexing starts at `a[1]` instead of `a[0]`, and ranges `i:j` include the last index.
- Vectorizing the code doesn't improve its speed.

A more exhaustive list is available at https://docs.julialang.org/en/v1/manual/noteworthy-differences/#Noteworthy-differences-from-Python.

### Compilation

When you run a chunk of code for the first time, it takes longer due to compilation. Don't be surprised, it is fundamental to Julia's performance, and the following runs are much faster.

### Imports

A package can be imported with the keyword `using ...`, which makes all of its functions available without prefix (similar to the Python code `from ... import *`). If you want to keep prefixes, use `import ...` instead.

Before a package can be imported, it must be installed. Here we install all the packages we will need during this tutorial. The following cell may take a while to run, especially with a slow internet connection.

In [ ]:
using Pkg
Pkg.update()
Pkg.add([
        "BenchmarkTools",
        "GLPK",
        "GraphPlot",
        "JuMP",
        "LightGraphs",
        "PyPlot",
])

## 1. The basics

### Variables and elementary operations

Variable assignment works as one would expect. Note that you can use LaTeX symbols by typing (for instance) "\beta" + `Tab` in the REPL or a Jupyter cell. This also works for indices when typing "u\\_1" + `Tab`.

In [ ]:
x = 1
ε = 0.1
u₁ = 10

Elementary operations are well summed up in https://juliadocs.github.io/Julia-Cheat-Sheet/. When comparing to Python, the main changes are boolean operators..

### Types

Like in Python, each variable in Julia has a type.

In [ ]:
a = 1
b = 1.
c = '1'
d = "1"
typeof(a), typeof(b), typeof(c), typeof(d)

Julia's typing system is dynamic, i.e. variables can change types, but performance can be increased by avoiding such changes and specifying the types when they are known in advance.

### Functions

The first function you need to master is for printing:

In [ ]:
println("Hello world!")

More generally, a function is defined with the following syntax:

In [ ]:
function addition(a, b)
    println("I am adding stuff")
    return a + b
end

If we want, we can specify the types of the inputs and output:

In [ ]:
function addition(a::Int, b::Int)::Int
    println("I am adding integers")
    return a + b
end

Note that there is still only one function `addition`, but it now has two "methods": one for integers, and one more generic.

In [ ]:
methods(addition)

This is linked to a key feature of Julia called *multiple dispatch*: the program will decide at runtime which function to apply depending on the type of the arguments.

In [ ]:
addition(1, 2)

In [ ]:
addition(1, 0.5)

As in Python, you can add optional and keyword arguments.

### Arrays

Arrays can be created and extended just like in Python, using square brackets:

In [ ]:
a = [3, 2]
push!(a, 1)
println(a)

However, if we know the type and size of the array in advance, it is better to allocate the memory first (with the `undef` object):

In [ ]:
a = Array{Int, 1}(undef, 3)

These random numbers are here because if we don't write any value in it, the array still contains whatever was in memory there before. We must therefore fill it:

In [ ]:
a[1], a[2], a[3] = 3, 2, 1
println(a)

The syntax `Array{T, d}` denotes an array with element type `T` and `d` dimensions (or axes). For $d=1$ and $d=2$ we have shortcuts:

In [ ]:
v = Vector{Float64}(undef, 2)
M = Matrix{Float64}(undef, 3, 2);

Vectorizing a function `f` in Julia is done by adding a dot after its name.

In [ ]:
function square(n::Int)::Int
    return n^2
end

In [ ]:
square.(a)

The same goes for elementary operators, except the dot comes before:

In [ ]:
a.*2

In [ ]:
a.*2 .== a.^2

Beyond arrays, Julia also offers built-in support for common data structures such as sets and dictionaries.

### Conditions and loops

Here is an example of `if`-`then`-`else` block:

In [ ]:
function test_sign(n::Int)::String
    if n > 0
        return "strictly positive"
    elseif n < 0
        return "strictly negative"
    else
        return "zero"
    end
end

Here is an example of `for` loop:

In [ ]:
function fibonacci(n::Int)::Int
    a, b = 0, 1
    for k = 1:n
        a, b = b, a + b
    end
    return a
end

Loops also allow us to define arrays by comprehension:

In [ ]:
powers = [2^k for k = 0:10]

Finally, here is an example of `while` loop:

In [ ]:
collatz(n::Int)::Int = (n%2 == 0) ? n÷2 : 3n+1

function collatz_iterations(n::Int)::Int
    it = 0
    while n > 1
        n = collatz(n)
        it += 1
    end
    return it
end

### Structures

Julia's notion of object is nothing more than a tuple with named components. We can define one like this:

In [ ]:
struct Point3D
    name::String
    x::Float64
    y::Float64
    z::Float64
end

To create an object, just apply its name as a function:

In [ ]:
p = Point3D("HOME", 1., 7., -13.)

The structures themselves contain no methods. However, we can define some by writing functions that take an input with the type we defined:

In [ ]:
function display(p::Point3D)
    name, x, y, z = p.name, p.x, p.y, p.z
    println("I am the point $name with coordinates ($x, $y, $z)")
end

In [ ]:
display(p)

### Plots

Plots can be generated with a Julia version of `matplotlib.pyplot`:

In [ ]:
using PyPlot
pygui(false)

In [ ]:
x = collect(1:0.2:10)
y = sin.(x)
plot(x, y, color="blue", label="curve")
scatter(x, y, color="red", label="points")
xlabel("x axis")
ylabel("y axis")
legend()
title("The sine function")
show()

### Optimization

The standard library for mathematical programming in Julia is called `JuMP`, a quick tutorial is available at https://jump.dev/JuMP.jl/stable/quickstart/.

### Miscellaneous

To  display the documentation (docstring) of a function, simply type `?` followed by its name in the REPL:

In [ ]:
?sin

Comments start with `#`

In [ ]:
# this is a comment

Error messages can be generated using the following function. You can also throw and catch more sophisticated exceptions.

In [ ]:
error("You made a serious mistake")

Functions that modify one or more of their arguments typically end with `!`:

In [ ]:
function add_one!(x::Vector{Int})
    n = length(x)
    for i = 1:n
        x[i] += 1
    end
end

The execution time of a function can be measured as follows:

In [ ]:
using BenchmarkTools

In [ ]:
Δt = @belapsed (M = rand(5, 5); N = M^2)

The Julia equivalent of Python's `None` is called `nothing`. The constant `NaN` also exists in Julia, it has type `Float64`.

## 2. Linear recurrent sequences

In this problem, we go through different ways of computing the terms of the sequence 

\begin{align*}
    x_n & = w_1 x_{n-1} + ... + w_d x_{n-d}
\end{align*}

whose initial values are

\begin{align*}
    x_1 & = y_1 \\
    &~\vdots \\
    x_d & = y_d
\end{align*}

As you have surely noticed by now, Jupyter notebooks can also accomodate $\LaTeX$ code.

### 2.1 Recursive computation

Implement a function `x_rec(w::Vector{Float64}, y::Vector{Float64}, n::Int)::Float64` computing $x_n$ recursively.

Do not use it for $n > 30$ or your computer will explode.

In [ ]:
@assert x_rec([1., 1.], [1., 1.], 10) ≈ 55.

Implement a function `x_loop` computing $x_n$ using a loop.

In [ ]:
@assert x_loop([1., 1.], [1., 1.], 10) ≈ 55.

Implement a function `build_M` constructing the matrix
\begin{align*}
M = \begin{pmatrix}
0 & 1 & 0 & \cdots & & 0 \\
0 & 0 & 1 & 0 & \cdots & 0 \\
  & & \ddots & \ddots & \ddots & \vdots\\
\vdots & & &  0 & 1 & 0 \\
& & & & 0 & 1 \\
w_1 & w_2 &  & \cdots &  & w_d
\end{pmatrix}
\end{align*}

In [ ]:
@assert build_M([1., 2., 3.]) ≈ [0. 1. 0. ; 0. 0. 1. ; 1. 2. 3.]

Implement a function `x_pow` computing $x_n$ using the powers of $M$.

In [ ]:
@assert x_pow([1., 1.], [1., 1.], 10) ≈ 55.

Implement a function `pow_eigen` computing the powers of a matrix using its eigendecomposition, assuming it exists. You can compute it using the library `LinearAlgebra`.

In [ ]:
using LinearAlgebra

In [ ]:
randM = rand(3, 3)
@assert pow_eigen(randM + randM', 5) ≈ (randM + randM')^5

Implement a function `x_eigen` computing $x_n$ using the eigendecomposition of $M$.

In [ ]:
@assert x_eigen([1., 1.], [1., 1.], 10) ≈ 55.

Plot and the execution times of all your functions for various values of $n$. Conclude on the fastest approach.

Bonus question: why do both methods using matrix powers have sublinear complexity?

## 3. Graph algorithms

### Data structure

Implement a weighted directed graph data structure called `Graph` where vertices are labelled from $1$ to $n$. Make sure that its attributes can be modified by putting the keyword `mutable` before `struct`.

### Describing a graph

Implement a function `nv(G)` counting the vertices of a graph

Implement a function `vertices(G)` listing the vertices of a graph

Implement a function `has_edge(G, u, v)` checking whether edge $(u, v)$ exists

Implement the functions `inneighbors(G, v)` and `outneighbors(G, u)`, which list the parents and children of a vertex respectively

Implement a function `edges(G)` listing the edges of a graph

Implement a function `weight(G, u, v)` computing the weight of an arc

You can use the following function to plot a small graph:

In [ ]:
import LightGraphs
using GraphPlot

function plot_graph(G::Graph)
    LG = LightGraphs.SimpleGraph(nb_vertices(G))
    for (u, v) in edges(G)
        add_edge!(LG, u, v)
    end
    plotg(LG, nodelabel=1:nv(LG))
end

### Modifying a graph

Implement functions `add_vertex!(G)` and `add_edge!(G, u, v, weight)`.

### Shortest paths

Implement the Ford-Bellman algorithm

Implement depth-first search and topological sorting within a function `dfs`

Implement Dijkstra's algorithm, either in a naive way or using the priority queue of `DataStructures`

### Spanning tree

Implement Kruskal's algorithm

### Flows and matchings

Implement the Edmunds-Karp algorithm, using edge weights as the capacities

Implement a function checking whether a graph is bipartite

Implement the Hungarian algorithm

### Test your functions on real graphs

## 4. Mathematical programming

Implement the LP formulation of the maximum flow problem

Implement the LP resolution method for the maximum spanning tree